In [1]:
import os
os.system('pip install scikit-learn sktime')
import numpy as np
from sktime.datasets import load_UCR_UEA_dataset, load_from_tsfile
from sklearn.preprocessing import StandardScaler
from ppt.metric import ACF_COS

Defaulting to user installation because normal site-packages is not writeable


In [2]:
def extract_data(data, label):
    res_data = []
    res_labels = []
    unique_label = np.unique(label)
    label_map = {label: i for i, label in enumerate(unique_label)}
    for index in range(0, len(data)):
        t_data = np.array([ d.tolist() for d in data.iloc[index]])
        t_label = label_map[label[index]]
        res_data.append(t_data)
        res_labels.append(t_label)
    return np.array(res_data).swapaxes(1, 2), np.array(res_labels)

In [3]:
train_raw_X, train_raw_y = load_from_tsfile(os.path.join('./dataset/CharacterTrajectories/CharacterTrajectories_eq_TRAIN.ts'))
test_raw_X, test_raw_y = load_from_tsfile(os.path.join('./dataset/CharacterTrajectories/CharacterTrajectories_eq_TEST.ts'))

train_X, train_y = extract_data(train_raw_X, train_raw_y)
test_X, test_y = extract_data(test_raw_X, test_raw_y)

print(f"Train X shape: {train_X.shape}")
print(f"Train y shape: {train_y.shape}")
print(f"Test X shape: {test_X.shape}")
print(f"Test y shape: {test_y.shape}")

scaler = StandardScaler()
scaler.fit(train_X.reshape(-1, train_X.shape[-1]))
train_X = scaler.transform(train_X.reshape(-1, train_X.shape[-1])).reshape(train_X.shape)
test_X = scaler.transform(test_X.reshape(-1, test_X.shape[-1])).reshape(test_X.shape)

# repeat the last element in sequence to make the sequence length 120. 
# train_X: (1422, 119, 3) -> (1422, 120, 3)
# test_X: (1436, 119, 3) -> (1436, 120, 3)
# this is only for CharacterTrajectories dataset 
train_X = np.concatenate([train_X, np.repeat(train_X[:, -1:, :], 1, axis=1)], axis=1)
test_X = np.concatenate([test_X, np.repeat(test_X[:, -1:, :], 1, axis=1)], axis=1)


train_X = train_X.transpose(0, 2, 1)
test_X = test_X.transpose(0, 2, 1)
# test_X = test_X[0: 1, 0: 2, :12]

seq_len = test_X.shape[2]
num_channels = test_X.shape[1]

print("test_X shape: ", test_X.shape)

Train X shape: (1422, 119, 3)
Train y shape: (1422,)
Test X shape: (1436, 119, 3)
Test y shape: (1436,)
test_X shape:  (1436, 3, 120)


In [4]:

patch_len = [3, 4, 6]
permute_freq = [10, 20, 40]

combinations = []
for p_len in patch_len:
    for p_freq in permute_freq:
        combinations.append((p_len, p_freq))

scores = []

for patch_len, permute_freq in combinations:
    acf_cos = ACF_COS(
        channel_num=num_channels,
        original_time_len=seq_len,
        patch_len=patch_len,
        permute_freq=permute_freq,
        device='cuda',
        permute_tensor_size=1000
    )

    score = acf_cos(test_X)

    mean = np.mean(score)
    std = np.std(score)
    mean_acf = np.mean(score)
    print(f'{patch_len:d} | {permute_freq:d}: {mean_acf:.2f}')


    scores.append(score)

Creating permutation tensor at shuffled_index/shuffled_index_random_10_3_03_40.pt, with strategy random, frequency 10, and patch length 3


Creating permutation tensor:   0%|          | 0/1000 [00:00<?, ?it/s]

Input ACF-COS data is 3D (Batch, Channel, Time), trying to spilt the Time dimension into (Patch Length, Patch Number)
3 | 10: 0.22
Creating permutation tensor at shuffled_index/shuffled_index_random_20_3_03_40.pt, with strategy random, frequency 20, and patch length 3


Creating permutation tensor:   0%|          | 0/1000 [00:00<?, ?it/s]

Input ACF-COS data is 3D (Batch, Channel, Time), trying to spilt the Time dimension into (Patch Length, Patch Number)
3 | 20: 0.39
Creating permutation tensor at shuffled_index/shuffled_index_random_40_3_03_40.pt, with strategy random, frequency 40, and patch length 3


...

Input ACF-COS data is 3D (Batch, Channel, Time), trying to spilt the Time dimension into (Patch Length, Patch Number)
6 | 10: 0.44
Creating permutation tensor at shuffled_index/shuffled_index_random_20_6_03_20.pt, with strategy random, frequency 20, and patch length 6


Creating permutation tensor:   0%|          | 0/1000 [00:00<?, ?it/s]

Input ACF-COS data is 3D (Batch, Channel, Time), trying to spilt the Time dimension into (Patch Length, Patch Number)
6 | 20: 0.58
Creating permutation tensor at shuffled_index/shuffled_index_random_40_6_03_20.pt, with strategy random, frequency 40, and patch length 6


Creating permutation tensor:   0%|          | 0/1000 [00:00<?, ?it/s]

Input ACF-COS data is 3D (Batch, Channel, Time), trying to spilt the Time dimension into (Patch Length, Patch Number)
6 | 40: 0.60
